In [12]:
import numpy as np
import pandas as pd
import time
from collections import defaultdict
from queue import PriorityQueue
import random
from copy import deepcopy
import math

In [13]:
# to reshape string into matrix
def printStringMatrix(state_type, state_characters):
    print(state_type)
    temp=state_characters[:]
    print(np.reshape([*temp],(3,3)))

# to take start state, target state as input [from user] and store the indexes of respective elements in target state using dictionary
def readInputs():
  sourceCharacters = input('Enter the source_state: ')
  targetCharacters = input('Enter the target_state: ')
  printStringMatrix('Source State', sourceCharacters)
  printStringMatrix('Target State', targetCharacters)

  h2Dictionary = {}
  for e in range(len(targetCharacters)):
    x = int(e/3)
    y = int(e%3)
    h2Dictionary[targetCharacters[e]] = (x,y)
    
  return sourceCharacters, targetCharacters, h2Dictionary

In [14]:
class individual:
  def __init__(self, board, genes = None):
    self.board = board
    self.genes = genes
  
  def cost(self, cost_func):
    board = deepcopy(self.board)

    #Apply the move function to the original board and return its cost function
    for i in self.genes:
      board = move(board, i)
    return cost_func(board,target_board)

In [15]:
# Heuristic 1-> h(n)= Tiles displaced ignoring Blank character tile
def h1Heuristic(intermediateCharacters, targetCharacters):
  cnt = 0
  for e in range(len(targetCharacters)):
    if intermediateCharacters[e] != '0' and intermediateCharacters[e] != targetCharacters[e]:
      cnt = cnt + 1
  return cnt
  
# Heuristic 2-> h(n)= Manhatten distance ignoring blank character tile
def h2Heuristic(intermediateCharacters, targetCharacters):
  dis = 0
  for e in range(len(intermediateCharacters)):
    if intermediateCharacters[e] != '0':
      x = int(e / 3)
      y = e % 3
      # Index of intermediateCharacters[e] in target
      _x = h2Dictionary[intermediateCharacters[e]][0]
      _y = h2Dictionary[intermediateCharacters[e]][1]
      dis = dis + abs(x -_x) + abs(y -_y)
  return dis

In [16]:
# accessing elements at (x-1,y), (x+1,y), (x,y-1) and (x,y+1) unless invalid
def goUp(idx):
    if idx > 2:
        return idx - 3
    return -1

def goDown(idx):
    if idx < 6:
        return idx + 3
    return -1

def goLeft(idx):
    if idx != 0 and idx != 3 and idx != 6:
        return idx -1
    return -1

def goRight(idx):
    if idx !=2 and idx != 5 and idx != 8:
        return idx + 1
    return -1

# traversing to valid states
def move(currentState, direction):
  cpy = currentState[:]
  for i in range(len(currentState)):
        if currentState[i] == '0':
            currentBlank=i
  if direction == 'u':
    idx = goUp(currentBlank)
  elif direction == 'd':
    idx = goDown(currentBlank)
  elif direction == 'r':
    idx = goRight(currentBlank)
  elif direction == 'l':
    idx = goLeft(currentBlank)
  if idx != -1:
    newBlank = idx
    newBlankCharacter = cpy[newBlank]
    cpy = cpy[:currentBlank] + cpy[newBlank] + cpy[currentBlank+1:]
    cpy = cpy[:newBlank] + '0' + cpy[newBlank+1:]
      
  return cpy

In [17]:
# Termination functions
def task_completed(cost_func):
  for i in population:
    if i.cost(cost_func) == 0:
      return True
  return False

In [18]:
# Crossover functions
def mate(a, b):
  if len(b.genes) > len(a.genes):
    temp = a
    a = b
    b = temp
  A = []
  B = []
  cross_over_len = random.randint(1, min(len(a.genes), len(b.genes)))
  A = b.genes[:cross_over_len] + a.genes[cross_over_len:]
  B = a.genes[:cross_over_len] + b.genes[cross_over_len:]
  return individual(a.board, A), individual(b.board, B)

def cross_over():
  num_cross_over = math.ceil(len(population)*CROSSOVER)
  parents = list(range(len(population)))
  for i in range(int(num_cross_over)):
      if len(parents) < 2:
        break
      a, b = random.sample(parents, 2)
      parents.remove(a)
      parents.remove(b)
      if len(population[a].genes)>0 and len(population[b].genes)>0 :
        A, B = mate(population[a], population[b])
        population.append(A)
        population.append(B)

In [19]:
#Selection functions
def spin_the_wheel(wheel_size, cost_invs):
  spin_val = random.uniform(0.0,1) * wheel_size
  for i, cost_inv_lim in enumerate(cost_invs):
    if spin_val < cost_inv_lim:
      return i

def next_gen_select(cost_func):
  cost_invs = []
  wheel_size = 0
  next_gen = []
  for p in population:
      # print(p.cost(cost_func))
      cost_inv = 1/(1e-6 + p.cost(cost_func))
      wheel_size += cost_inv
      cost_invs.append(wheel_size)
  while len(next_gen) < INIT_POPULATION:
      i = spin_the_wheel(wheel_size, cost_invs)
      next_gen.append(population[i])
  return next_gen

In [20]:
#Mutation functions
def perform_mutations():
  for p in population:
    if random.random() < MUTATION:
      mutate(p)

def mutate(child):
  if (not child.genes or random.random() < 0.6):
    temp = [1,1,1,1]
    for i in range(4):
      move = ['u','d','r','l']
      temp[i] = deepcopy(child)
      temp[i].genes.append(move[i])
      population.append(temp[i])      
  else:
      i = random.randint(0, len(child.genes)-1)
      child.genes[i] = random.choice(['u','d','r','l'])

In [21]:
def perform_genetic_algorithm(init_board,target_board, FITNESS_FUNCTION):
  
  iteration = 0
  while iteration < MAX_ITER and not task_completed(FITNESS_FUNCTION):
      if task_completed(FITNESS_FUNCTION):
        break
      cross_over()
      perform_mutations()
      population = next_gen_select(FITNESS_FUNCTION)
      iteration += 1

  if task_completed(FITNESS_FUNCTION):
    for p in population:
      if p.cost(FITNESS_FUNCTION) == 0:
        print("SUCCESS")
        print("Answer found in "+str(iteration+1)+" generations")
        print("Seq of actions required: ", p.genes)
        print("STEPS:")
        board = p.board
        printStringMatrix('Initial Board', board)
        for i in p.genes:
          board = move(board, i)
          printStringMatrix('Moving '+i, board)
        break
  else:
    print("FAILURE")

In [23]:
# init_board =  413726058 # solvable
# init_board =  013425786 # solvable
init_board,target_board,h2Dictionary=readInputs()

#parameters
MAX_ITER = 5000
INIT_POPULATION = 10
CROSSOVER = 0.6
MUTATION = 0.2
_population = [individual(init_board, list()) for _ in range(INIT_POPULATION)]

print('\n\nH1 Heuristic Result:')
population = deepcopy(_population)
perform_genetic_algorithm(init_board,target_board, h1Heuristic)


print('\n\nH2 Heuristic Result:')
population = deepcopy(_population)
perform_genetic_algorithm(init_board,target_board,h2Heuristic)


Enter the source_state: 413726058
Enter the target_state: 123456780
Source State
[['4' '1' '3']
 ['7' '2' '6']
 ['0' '5' '8']]
Target State
[['1' '2' '3']
 ['4' '5' '6']
 ['7' '8' '0']]


H1 Heuristic Result:
SUCCESS
Answer found in 6 generations
Seq of actions required:  ['d', 'l', 'u', 'u', 'u', 'r', 'u', 'u', 'd', 'd', 'r']
STEPS:
Initial Board
[['4' '1' '3']
 ['7' '2' '6']
 ['0' '5' '8']]
Moving d
[['4' '1' '3']
 ['7' '2' '6']
 ['0' '5' '8']]
Moving l
[['4' '1' '3']
 ['7' '2' '6']
 ['0' '5' '8']]
Moving u
[['4' '1' '3']
 ['0' '2' '6']
 ['7' '5' '8']]
Moving u
[['0' '1' '3']
 ['4' '2' '6']
 ['7' '5' '8']]
Moving u
[['0' '1' '3']
 ['4' '2' '6']
 ['7' '5' '8']]
Moving r
[['1' '0' '3']
 ['4' '2' '6']
 ['7' '5' '8']]
Moving u
[['1' '0' '3']
 ['4' '2' '6']
 ['7' '5' '8']]
Moving u
[['1' '0' '3']
 ['4' '2' '6']
 ['7' '5' '8']]
Moving d
[['1' '2' '3']
 ['4' '0' '6']
 ['7' '5' '8']]
Moving d
[['1' '2' '3']
 ['4' '5' '6']
 ['7' '0' '8']]
Moving r
[['1' '2' '3']
 ['4' '5' '6']
 ['7' '8' '0']]